In [ ]:
%pip install fastai matplotlib 

In [ ]:
%reload_ext autoreload
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.metrics import error_rate,accuracy
import pandas as pd
import os

Path = os.path

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!mkdir Dataset

In [ ]:
!unzip -I "utf-8" "/content/gdrive/My Drive/state-farm-distracted-driver-detection.zip" -d "/content/Dataset"

In [ ]:
!mkdir "/content/Images"
!unzip -I "utf-8" "/content/Dataset/imgs.zip" -d "/content/Dataset/Images"

In [ ]:
df = pd.read_csv("/content/Dataset/driver_imgs_list.csv")
df.head()

In [ ]:
path = Path("/content/Dataset/Images")
train_set = Path("/content/Dataset/Images/train")
test_set = Path("/content/Dataset/Images/test")

In [ ]:
dataset = (ImageList.from_folder(train_set)
                                .split_by_rand_pct(0.2)
                                .label_from_folder()
                                .transform(get_transforms(), size=256)
                                .add_test_folder(test_set)).databunch(bs=32).normalize(imagenet_stats)

In [ ]:
dataset.classes

In [ ]:
dataset.show_batch(3)

In [ ]:
learner = cnn_learner(dataset, models.resnet34, metrics=[accuracy,error_rate])

In [ ]:
learner.fit_one_cycle(1)

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(8, slice(0.001))

In [ ]:
learner.save('stage-1')

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)

In [ ]:
interp.plot_top_losses(6)

In [ ]:
learner.load('stage-1')

In [ ]:
interp.most_confused(min_val=1)

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(10, slice(1e-04, 1e-03))

In [ ]:
learner.save('stage-2');

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)

In [ ]:
interp.plot_top_losses(6)

In [ ]:
interp.most_confused(min_val=1)

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
count=0;
Dict = {0: 'safe driving', 1: 'texting - right', 2: 'talking on the phone - right', 3: 'texting - left', 4: 'talking on the phone - left', 5: 'operating the radio', 6: 'drinking', 7: 'reaching behind', 8: 'hair and makeup', 9: 'talking to passenger'}
for dirname, _, filenames in os.walk('/content/Dataset/Images/test'):
  for filename in filenames:
    count+=1;
    img = open_image(os.path.join(dirname, filename))
    pred_class,pred_index,outputs = learner.predict(img)
    i = str(pred_class)
    print(Dict.get(int(i[1])))
    img1=mpimg.imread(os.path.join(dirname, filename))
    imgplot = plt.imshow(img1)
    plt.show()
    if(count>10):
        break